# 3-class classification solution

Start with prepare the dataset:

In [1]:
from cProfile import label

import pandas as pd

df = pd.read_csv("../../data/processed_data.csv")

def get_value(row: "a row of dataframe"):
    if row['winner_model_a'] == 1:
        return 0
    elif row['winner_model_b'] == 1:
        return 1
    else:
        return 2

df["label"] = df.apply(get_value, axis = 1)     # axis=1: apply to every ROW

def gen_input(row):
    return f"[PROMPT] {row['clean_prompt']} [RESPONSE_A] {row['clean_response_a']} [RESPONSE_B] {row['clean_response_b']}"

df["input"] = df.apply(gen_input, axis=1)
df[["label","input"]]

,label,input
0,0,[PROMPT] is it morally right to try to have a ...
1,1,[PROMPT] what is the difference between marria...
2,2,[PROMPT] explain function calling. how would y...
3,0,[PROMPT] how can i create a test set for a ver...
4,1,[PROMPT] what is the best way to travel from t...
...,...,...
57472,0,[PROMPT] a simple mnemonic for π how i wish i ...
57473,0,"[PROMPT] in python, implement a naive bayes wi..."
57474,0,[PROMPT] is it unethical to work on building w...
57475,1,"[PROMPT] if a bait contains 0, 0025 bromadiolo..."


Then we apply the model from transformer.

First, we tokenize our input information, which means making them into numbers that BERT can understand.

In [2]:
from transformers import BertTokenizer

tknzr = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenize
encodings = tknzr(
    df["input"].tolist(),
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt" # pytorch tensors
)

D:\Projects\UCI-Datathon-2025\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Second, we create dataset object for pytorch:

In [3]:
import torch
from torch.utils.data import Dataset

class LLMPDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings  # input_ids and attention_mask
        self.labels = labels

    def __getitem__(self, i):
        item = {key: val[i] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[i])
        return item

    def __len__(self):
        return len(self.labels)

dst = LLMPDataset(encodings, df['label'].tolist())

Load a pre-trained model:

In [4]:
from transformers import BertForSequenceClassification

# use the model with 3-output labels
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Set trainning arguments:

In [5]:
import accelerate
print(accelerate.__version__)

0.26.0


In [10]:
from transformers import TrainingArguments
import accelerate

train_arg = TrainingArguments(
    output_dir="./train_result",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,  # Mixed precision for speed
)

Then, train the model:

In [11]:
import torch
print(torch.cuda.is_available())  # should return True
print(torch.cuda.get_device_name(0))  # prints your GPU name if available

True
NVIDIA GeForce RTX 4060 Laptop GPU


In [12]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args= train_arg,
    train_dataset= dst,
    eval_dataset= dst
)

trainer.train()

D:\Projects\UCI-Datathon-2025\.venv\lib\site-packages\accelerate\accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,1.100600,1.097652
2,1.093500,1.092521


TrainOutput(global_step=7186, training_loss=1.0975880366728028, metrics={'train_runtime': 3102.5845, 'train_samples_per_second': 37.051, 'train_steps_per_second': 2.316, 'total_flos': 3.024593982150451e+16, 'train_loss': 1.0975880366728028, 'epoch': 2.0})

Evaluate the result:

In [13]:
results = trainer.evaluate()
print(results)

{'eval_loss': 1.0925211906433105, 'eval_runtime': 303.8905, 'eval_samples_per_second': 189.137, 'eval_steps_per_second': 11.823, 'epoch': 2.0}


In [16]:
import numpy as np
from sklearn.metrics import accuracy_score

pred_output = trainer.predict(dst)

logits = pred_output.predictions
preds = np.argmax(logits, axis=1)
labels = pred_output.label_ids

accuracy = accuracy_score(labels, preds)
print("Accuracy on full dataset:", accuracy)

Accuracy on full dataset: 0.36357151556274686
